[Reference](https://python.plainenglish.io/pydantic-is-a-game-changer-40d632518c35)

# 1. Type checking and detailed error messages without boilerplate code

In [1]:
class User:
    def __init__(self, name, age, is_active):
        assert isinstance(name, str), "name must be a string"
        assert isinstance(age, int), "age must be an integer"
        assert isinstance(is_active, bool), "is_active must be a boolean"
        self.name = name
        self.age = age
        self.is_active = is_active

try:
    user = User(name='Alice', age='thirty', is_active=True)
except Exception as e:
    print(e)

age must be an integer


In [2]:
from pydantic import BaseModel, ValidationError
class User(BaseModel):
    name: str
    age: int
    is_active: bool

try:
    user = User(name='Alice', age='thirty', is_active=True)
except ValidationError as e:
    print(e)

1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/int_parsing


# 2. Data serialization and deserialization of complex object is easy!

In [3]:
import json

class Item:
    def __init__(self, name, price):
        self.name = name
        self.price = price

class Order:
    def __init__(self, id, items):
        self.id = id
        self.items = items

    def to_json(self):
        return json.dumps({
            'id': self.id,
            'items': [{'name': item.name, 'price': item.price} for item in self.items]
        })

# Manual serialization to JSON
order = Order(id=123, items=[Item('Widget', 34.99)])
print(order.to_json())

# Manual deserialization from JSON
data = {"id": 123, "items": [{"name": "Widget", "price": 34.99}]}
items = [Item(**x) for x in data['items']]
order = Order(id = data['id'],items = items)

{"id": 123, "items": [{"name": "Widget", "price": 34.99}]}


In [4]:
from pydantic import BaseModel
from typing import List

class Item(BaseModel):
    name: str
    price: float

class Order(BaseModel):
    id: int
    items: List[Item]

# Serializes the object to a JSON string
order = Order(id=123, items=[{'name': 'Widget', 'price': 34.99}])
print(order.model_dump_json())

# Deserializes the object from a JSON string
data = {"id": 123, "items": [{"name": "Widget", "price": 34.99}]}
order = Order(**data)

{"id":123,"items":[{"name":"Widget","price":34.99}]}


# 3. Automatic schema documentation

In [5]:
print(Order.schema_json(indent=2))

{
  "$defs": {
    "Item": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        },
        "price": {
          "title": "Price",
          "type": "number"
        }
      },
      "required": [
        "name",
        "price"
      ],
      "title": "Item",
      "type": "object"
    }
  },
  "properties": {
    "id": {
      "title": "Id",
      "type": "integer"
    },
    "items": {
      "items": {
        "$ref": "#/$defs/Item"
      },
      "title": "Items",
      "type": "array"
    }
  },
  "required": [
    "id",
    "items"
  ],
  "title": "Order",
  "type": "object"
}
